In [2]:
from ga_objects.problem import ProblemSingleElementWise
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from ga_objects.callback import UpdatePopulationCallback
from utils.encoder import Encoder
from utils.tools import Tools
import warnings
import logging
from ga_objects.terminator import MyTermination
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.termination.default import DefaultSingleObjectiveTermination, DefaultMultiObjectiveTermination
from utils.testSetup import Setup
from pymoo.core.duplicate import ElementwiseDuplicateElimination
from ga_objects.problem import ProblemMultiElementWise
import numpy as np
from scipy.spatial.distance import cdist
import pandas as pd
import pm4py
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareAnalyzer import MPDeclareAnalyzer
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareResultsBrowser import MPDeclareResultsBrowser

warnings.filterwarnings("ignore", ".*feasible.*")
logging.getLogger('matplotlib').setLevel(logging.WARNING)

DEBUG:matplotlib:CACHEDIR=C:\Users\matte\.matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from C:\Users\matte\.matplotlib\fontlist-v390.json


In [4]:
from Declare4Py.D4PyEventLog import D4PyEventLog
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareAnalyzer import MPDeclareAnalyzer
from ga_objects.problem import ProblemSingleElementWise
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from ga_objects.callback import UpdatePopulationCallback

from utils.encoder import Encoder
from utils.tools import Tools
import warnings
import logging
from ga_objects.terminator import MyTermination
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.termination.default import DefaultSingleObjectiveTermination, DefaultMultiObjectiveTermination
from utils.testSetup import Setup
from pymoo.core.duplicate import ElementwiseDuplicateElimination
from ga_objects.problem import ProblemMultiElementWise
import numpy as np
from scipy.spatial.distance import cdist
import pandas as pd
import pm4py
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareAnalyzer import MPDeclareAnalyzer
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareResultsBrowser import MPDeclareResultsBrowser


xes_file = "../declare_models/model4/log_50.xes"
save_path = "../declare_models/model4_initial_pop.csv"
event_log = pm4py.read_xes(xes_file)
event_log.to_csv(save_path, index=False)

pop_size = 3000
trace_length = 50

# feasible_init_pop = Setup.initial_population("../declare_models/model1_initial_pop.csv",trace_length=50,trace_number=10)
feasible_init_pop = Setup.extract_traces("../declare_models/model3_initial_pop.csv")


encoder, d4py, event_log, dataframe, initial_encoded_pop, lower_bounds, upper_bounds = Setup.initialize_shared_components(
            path_to_declareModel="../declare_models/model2/model2.decl",
            trace_length=50,
            initial_population=feasible_init_pop
        )

timestamps = Tools.generate_random_timestamps(trace_length)

traces = Tools.load_traces_from_csv("results/03-24-20-44/encoded_traces/ID_1_run_1_ProblemSingle.csv")
data = {
            'case:concept:name': ['1'] * trace_length,
            'concept:name': feasible_init_pop[5],
            'timestamp': pd.to_datetime(timestamps),
}
dataframe = pd.DataFrame(data)

event_log = D4PyEventLog()
event_log.log = dataframe
event_log.timestamp_key = "timestamp"
event_log.activity_key = "concept:name"
event_log.to_eventlog()

basic_checker = MPDeclareAnalyzer(log=event_log, declare_model=d4py, consider_vacuity=True)
conf_check_res: MPDeclareResultsBrowser = basic_checker.run()

print(conf_check_res.get_metric(trace_id=0, metric="state"))
print(len(conf_check_res.get_metric(trace_id=0, metric="state")))
print("-------------------------------------------")
print(d4py.constraints[-4])

# Tools.calculate_overall_diversity(encoder,"results/03-26-21-45/encoded_traces/ID_3_run_1_ProblemSingle.csv",50)


# Tools.constraints_eval(dataframe,event_log,d4py,feasible_init_pop)


parsing log, completed traces :: 100%|██████████| 10/10 [00:00<00:00, 1074.97it/s]


FileNotFoundError: [Errno 2] No such file or directory: '../declare_models/model3_initial_pop.csv'

In [2]:
from pymoo.operators.crossover.pntx import TwoPointCrossover
from ga_objects.terminator import DiversityTermination
from pymoo.operators.crossover.sbx import SBX
from ga_objects.mutation import IntegerPolynomialMutation
from ga_objects.sampling import MySampling
from ga_objects.problem import ProblemSingle, ProblemSingleTrace
from utils.testSetup import Setup
from utils.tools import Tools
from ga_objects.problem import ProblemSingleElementWise
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from ga_objects.callback import UpdatePopulationCallback
from utils.tools import Tools
import warnings
import logging
from ga_objects.terminator import MyTermination
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.termination.default import DefaultSingleObjectiveTermination, DefaultMultiObjectiveTermination
from utils.testSetup import Setup
from pymoo.core.duplicate import ElementwiseDuplicateElimination
from ga_objects.problem import ProblemMultiElementWise
import numpy as np
from scipy.spatial.distance import cdist
import pandas as pd
from ga_objects.algorithm import CustomNSGA2

warnings.filterwarnings("ignore", ".*feasible.*")
logging.getLogger('matplotlib').setLevel(logging.WARNING)


pop_size = 300
trace_length = 50

# feasible_init_pop = Setup.extract_traces("../declare_models/model4/initial_pop_90.csv", trace_length)

random_pop = Setup.initial_random_population("../declare_models/model4/model4.decl",50,300)
# feasible_init_pop = Tools.extract_traces_from_xes("../declare_models/log_model2_refinedModel.xes", 50)
#
encoder, d4py, event_log, dataframe, initial_encoded_pop, lower_bounds, upper_bounds = Setup.initialize_shared_components(
            path_to_declareModel="../declare_models/model4/model4.decl",
            trace_length=trace_length,
            initial_population=random_pop
        )

problem = ProblemSingleTrace(
            trace_length=trace_length,
            encoder=encoder,
            d4py=d4py,
            initial_population=initial_encoded_pop,
            xl=lower_bounds,
            xu=upper_bounds,
            event_log=event_log,
            dataframe=dataframe,
)

# initialize the algorithm based on the arguments
algorithm = GA(
    problem=problem,
    pop_size=pop_size,
    sampling=MySampling(initial_population=initial_encoded_pop),
    crossover=TwoPointCrossover(prob=0.9),
    mutation=IntegerPolynomialMutation(prob=0.5, eta=1),
    eliminate_duplicates=True,
)



termination = DefaultMultiObjectiveTermination(
            xtol=1e-8, cvtol=1e-6, ftol=0.0025, period=30,
            n_max_gen=50000, n_max_evals=100000
        )

termination2 = DiversityTermination(0.95, 200)
termination3 = MyTermination(1,"F",0)


result = minimize(problem, algorithm, termination=termination3, verbose=True)




# encoded_traces = [traces]
# population = np.array(encoded_traces)

# diversity_score = np.mean(cdist(trace_path, trace_path, metric='hamming'))

# print(diversity_score)
# Tools.calculate_overall_diversity(encoder, trace_path, 90)


n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |      300 |  2.256000E+01 |  1.400000E+01
     2 |      600 |  2.115333E+01 |  1.400000E+01
     3 |      900 |  2.003667E+01 |  1.400000E+01
     4 |     1200 |  1.901667E+01 |  1.400000E+01
     5 |     1500 |  1.777667E+01 |  1.400000E+01
     6 |     1800 |  1.619333E+01 |  1.400000E+01
     7 |     2100 |  1.502667E+01 |  1.300000E+01
     8 |     2400 |  1.449333E+01 |  1.200000E+01
     9 |     2700 |  1.407333E+01 |  1.200000E+01
    10 |     3000 |  1.372667E+01 |  1.100000E+01
    11 |     3300 |  1.343333E+01 |  1.100000E+01
    12 |     3600 |  1.293667E+01 |  1.100000E+01
    13 |     3900 |  1.270333E+01 |  1.100000E+01
    14 |     4200 |  1.233667E+01 |  1.000000E+01
    15 |     4500 |  1.188000E+01 |  1.000000E+01
    16 |     4800 |  1.164333E+01 |  9.0000000000
    17 |     5100 |  1.135000E+01 |  9.0000000000
    18 |     5400 |  1.084333E+01 |  9.0000000000
    19 |     5700 |  1.059333E+01 |  9.0000000000


KeyboardInterrupt: 

In [4]:
from scipy.spatial.distance import pdist, squareform

pop_size = 4000
trace_length = 70


feasible_init_pop = Setup.extract_traces("../declare_models/model4/initial_pop_70.csv", trace_length)
# feasible_init_pop = Tools.extract_traces_from_xes("../declare_models/log_model2_refinedModel.xes", 50)
#
encoder, d4py, event_log, dataframe, initial_encoded_pop, lower_bounds, upper_bounds = Setup.initialize_shared_components(
            path_to_declareModel="../declare_models/model4/model4.decl",
            trace_length=trace_length,
            initial_population=feasible_init_pop
        )

file_path = "results/04-16-01-09/encoded_traces/ID_1_run_1_ProblemMultiObjectiveNovelty.csv"

df = pd.read_csv(file_path, header=None, delimiter=";")
traces = df.values.tolist()
encoded_traces = [encoder.encode(trace) for trace in feasible_init_pop]


population = np.array(encoded_traces)

Tools.calculate_overall_diversity(encoder,file_path,trace_length)

dist_matrix = cdist(population,population, metric='hamming')

print(dist_matrix.mean())

dist_matrix = squareform(pdist(population, metric="hamming"))
n = dist_matrix.shape[0]
mean_per_trace = (np.sum(dist_matrix, axis=1) - 0) / (n - 1)
print(len(dist_matrix))
print(mean_per_trace.mean())

dist_matrix = pdist(population, metric='hamming')
print(len(dist_matrix))
print(dist_matrix.mean())



Overall Diversity Score: 0.8328546821428571
0.6045714285714286
10
0.6717460317460319
45
0.6717460317460318
